In [1]:
import os
import pandas as pd
import numpy as np
import io
import vcf
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def read_split_vcfs(path,run):
    """Parse annotated vcfs that have been processed with bcftools split-vep.
    Return dataframe of vcfs.
    
    Arguments:
    path -- path to split vcf files
    run -- Run identifier (str)
    """

    df = pd.DataFrame(columns=['Run','Sample',"CHROM","POS","ID","END","CSQ_SYMBOL","CSQ_Feature", "CSQ_EXON", "CSQ_INTRON","QUAL", 
                               "GT","CN","NP","QA","QS","QSE","QSS",
                               "REF","ALT","CSQ_VARIANT_CLASS", "CSQ_Consequence", "CSQ_IMPACT","CSQ_STRAND"])
    directory = os.fsencode(path)
    
    for file in os.listdir(directory):
        
        filename = os.fsdecode(file)
        
        
        if filename.endswith(".split.vcf"):
            
            vcf_reader = vcf.Reader(open((path+filename), 'r'))
            sample=filename.replace(".split.vcf","")

            for record in vcf_reader:

                entry = pd.DataFrame.from_dict({
                    "Run": run,
                    "Sample": filename,
                    "CHROM": record.CHROM,
                    "POS": record.POS,
                    "ID":record.ID,
                    "REF":record.REF,
                    "ALT":record.ALT,
                    "QUAL":record.QUAL,
                    "END":record.INFO["END"],
                    "CSQ_SYMBOL":record.INFO["CSQ_SYMBOL"],
                    "CSQ_Feature":record.INFO["CSQ_Feature"],
                    "CSQ_EXON":record.INFO["CSQ_EXON"], 
                    "CSQ_INTRON":record.INFO["CSQ_INTRON"], 
                    "GT":record.genotype(sample)['GT'],
                    "CN":record.genotype(sample)['CN'],
                    "NP":record.genotype(sample)['NP'],
                    "QA":record.genotype(sample)['QA'],
                    "QS":record.genotype(sample)['QS'],
                    "QSE":record.genotype(sample)['QSE'],
                    "QSS":record.genotype(sample)['QSS'],
                    "CSQ_VARIANT_CLASS":record.INFO["CSQ_VARIANT_CLASS"], 
                    "CSQ_IMPACT":record.INFO["CSQ_IMPACT"], 
                    "CSQ_STRAND":record.INFO["CSQ_STRAND"],
                    "CSQ_Consequence":record.INFO["CSQ_Consequence"], 
    
                    })

                df = pd.concat([df, entry], ignore_index=True,sort=False)
                
    return df

In [3]:
def read_segment_vcfs(path,run):
    """ Parse unannotated segment vcfs from gCNV.
    Return dataframe of vcfs.
    
    Arguments:
    path -- path to split vcf files
    run -- Run identifier (str)
    """

    #  Create dataframe
    df = pd.DataFrame(columns=['Run','Sample',"CHROM","POS","ID","REF","ALT","QUAL","END","GT","CN","NP","QA","QS","QSE","QSS"])
    
    # Get path into variable
    directory = os.fsencode(path)
    
    # Loop through files in the directory
    for file in os.listdir(directory):
        
        # Extract name of file
        filename = os.fsdecode(file)
        
        # Ensure only segments vcfs (merged intervals) are parsed
        if filename.endswith("_segments.vcf"):
            
            vcf_reader = vcf.Reader(open((path+filename), 'r'))
            sample=filename.replace("_segments.vcf","")

            # Populate dataframe (using pyvcf variables)
            for record in vcf_reader:

                entry = pd.DataFrame.from_dict({
                    "Run": run,
                    "Sample": filename,
                    "CHROM": record.CHROM,
                    "POS": record.POS,
                    "ID":record.ID,
                    "REF":record.REF,
                    "ALT":record.ALT,
                    "QUAL":record.QUAL,
                    "END":record.INFO["END"],
                    "GT":record.genotype(sample)['GT'],
                    "CN":record.genotype(sample)['CN'],
                    "NP":record.genotype(sample)['NP'],
                    "QA":record.genotype(sample)['QA'],
                    "QS":record.genotype(sample)['QS'],
                    "QSE":record.genotype(sample)['QSE'],
                    "QSS":record.genotype(sample)['QSS'],    
                    })
                
                df = pd.concat([df, entry], ignore_index=True,sort=False)
                
    return df

In [ ]:
# Create per run annotated dataframes for the three validation runs
annotated39=read_split_vcfs("/opt/notebooks/0039/","0039")
annotated42=read_split_vcfs("/opt/notebooks/0042/","0042")
annotated50=read_split_vcfs("/opt/notebooks/0050/","0050")

In [ ]:
# Create per run unannotated dataframes the three validation runs
raw39=read_segment_vcfs("/opt/notebooks/0039/unannotated/","0039")
raw42=read_segment_vcfs("/opt/notebooks/0042/unannotated/","0042")
raw50=read_segment_vcfs("/opt/notebooks/0050/unannotated/","0050")

In [ ]:
# Create one dataframe with all CNV calls 
all_cnv_calls=pd.concat([raw39,raw42,raw50],ignore_index=True)

# Sort dataframe first by chromosome then position
all_cnv_calls.sort_values(by=['CHROM', 'POS'])

# Setting to view all lines
pd.set_option("display.max_rows", None, "display.max_columns", None)

# Couldn't figure out the type of None so change type to str to be able to filter no-calls
all_cnv_calls["ALT"] = all_cnv_calls["ALT"].astype(str)
filtered_calls=all_cnv_calls[(all_cnv_calls["ALT"] != "None")].sort_values(by=['CHROM', 'POS'])

# Save all CNV calls to a file
filtered_calls.to_csv("All_Validation_CNVs_unannotated-220929.csv",index=False)


In [ ]:
# Save per-run dataframes to files.
annotated39.to_csv("0039-Annotated-CNV-Summary-220929.csv",index=False)
annotated42.to_csv("0042-Annotated-CNV-Summary-220802.csv",index=False)
annotated50.to_csv("0050-Annotated-CNV-Summary-220929.csv",index=False)

In [ ]:
# Make a all annotation dataframe
all_annotated_cnvs=pd.concat([annotated39,annotated42,annotated50])

# Save to file
all_annotated_cnvs.to_csv("All_Validation_CNVs_annotated-220802.csv",index=False)

## Create additional summaries as required

In [ ]:
# Create summaries for the minimum sample investifation
annotated39_32=read_split_vcfs("/opt/notebooks/0039-32Samples/","0039")
annotated39_32.to_csv("32sample_cohort_Annotated-CNV-Summary-220812.csv",index=False)

In [ ]:
# Add length to all CNVs from the three validation runs
filtered_calls["Length"] = filtered_calls["END"] - filtered_calls["POS"]

In [ ]:
# Read information about expected calls
CENCNV_samples=pd.read_csv('/opt/notebooks/220802_concordant_details.csv',',')

CENCNV_samples.sort_values(by='ID')

In [12]:
# Create summary for the sex ratio investigations
annotated42_sex=read_split_vcfs("/opt/notebooks/sex-ratio/42-5m-27F/","0042")
annotated42_sex.to_csv("0042-5M-27F-annotated_summary_220822.csv",index=False)

annotated39_sex=read_split_vcfs("/opt/notebooks/sex-ratio/39-5m-27F/","0039")
annotated39_sex.to_csv("0039-5M-27F-annotated_summary_220822.csv",index=False)


In [4]:
# Create per run unannotated dataframes
raw35=read_segment_vcfs("/opt/notebooks/0035/unannotated/","0035")
raw39=read_segment_vcfs("/opt/notebooks/0039/unannotated/","0039")
raw40=read_segment_vcfs("/opt/notebooks/0040/unannotated/","0040")
raw42=read_segment_vcfs("/opt/notebooks/0042/unannotated/","0042")
raw45=read_segment_vcfs("/opt/notebooks/0045/unannotated/","0045")
raw50=read_segment_vcfs("/opt/notebooks/0050/unannotated/","0050")
raw53=read_segment_vcfs("/opt/notebooks/0053/unannotated/","0053")
raw60=read_segment_vcfs("/opt/notebooks/0060/unannotated/","0060")
raw58=read_segment_vcfs("/opt/notebooks/0058/unannotated/","0058")
raw62=read_segment_vcfs("/opt/notebooks/0062/unannotated/","0062")
raw65=read_segment_vcfs("/opt/notebooks/0065/unannotated/","0065")

all_runs=pd.concat([raw35,raw39,raw40,raw42,raw45,raw50,raw53,raw58,raw60,raw62,raw65],ignore_index=True)

all_runs.to_csv('11-CEN-Runs.csv',index=False)

In [10]:
# positive only 0042 summary downsampled down to 15M
positives_42_15m=read_split_vcfs("/opt/notebooks/downsampling/0042/0015/","0042")
all_42_15M=read_segment_vcfs("/opt/notebooks/downsampling/0042/0015/unannotated/","0042")

positives_42_15m.to_csv('0042-15M-positives.csv',index=False)
all_42_15m.to_csv('0042-15M-all.csv',index=False)